In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Post-Traumatic_Stress_Disorder"
cohort = "GSE199841"

# Input paths
in_trait_dir = "../../input/GEO/Post-Traumatic_Stress_Disorder"
in_cohort_dir = "../../input/GEO/Post-Traumatic_Stress_Disorder/GSE199841"

# Output paths
out_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/GSE199841.csv"
out_gene_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/gene_data/GSE199841.csv"
out_clinical_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/clinical_data/GSE199841.csv"
json_path = "../../output/preprocess/Post-Traumatic_Stress_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression signatures in patients with posttraumatic stress disorder (PTSD) and healthy controls"
!Series_summary	"To investigate gene expression signatures in PTSD, we used microarray and performed blood-based transcriptome profiling. Our analyses identified key genes and molecular pathways that can be centrally involved in the etiology of PTSD."
!Series_overall_design	"The blood sampling was performed around noon (before lunch), between 11:30 AM and 12:30 PM. Gene expression in blood was measured for 32 women with PTSD and 16 healthy control women."
Sample Characteristics Dictionary:
{0: ['gender: female'], 1: ['age(years): 55', 'age(years): 53', 'age(years): 22', 'age(years): 24', 'age(years): 27', 'age(years): 44', 'age(years): 41', 'age(years): 21', 'age(years): 42', 'age(years): 51', 'age(years): 43', 'age(years): 56', 'age(years): 26', 'age(years): 31', 'age(years): 45', 'age(years): 39', 'age(years): 38', 'age(years): 47', 'age(years)

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the Series_summary, this dataset contains microarray transcriptome profiling
is_gene_available = True

# 2.1 Data Availability
# From the sample characteristics dictionary, we can identify the keys:
trait_row = 2  # 'diagnostic group' indicates PTSD status
age_row = 1    # Contains age information
gender_row = None  # This is constant (all female) so not useful for association study

# 2.2 Data Type Conversion functions
def convert_trait(value):
    """Convert PTSD trait status to binary values."""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'ptsd patient':
        return 1
    elif value.lower() == 'healthy control':
        return 0
    else:
        return None

def convert_age(value):
    """Convert age to numerical values."""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract just the numerical part
    age_str = ''.join(c for c in value if c.isdigit())
    if age_str:
        return int(age_str)
    else:
        return None

def convert_gender(value):
    """Convert gender to binary values."""
    # This function is defined for completeness but won't be used
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# 3. Save Metadata
# Initial filtering on data availability
# Trait is available since trait_row is not None
is_trait_available = trait_row is not None

validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait data is available, extract clinical features
if trait_row is not None:
    # Get clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    clinical_preview = preview_df(clinical_features)
    print("Clinical Features Preview:")
    print(clinical_preview)
    
    # Save clinical data to CSV
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Features Preview:
{'GSM5988345': [0.0, 55.0], 'GSM5988346': [0.0, 53.0], 'GSM5988347': [0.0, 22.0], 'GSM5988348': [0.0, 24.0], 'GSM5988349': [0.0, 27.0], 'GSM5988350': [0.0, 44.0], 'GSM5988351': [0.0, 41.0], 'GSM5988352': [0.0, 21.0], 'GSM5988353': [0.0, 27.0], 'GSM5988354': [0.0, 27.0], 'GSM5988355': [0.0, 42.0], 'GSM5988356': [0.0, 22.0], 'GSM5988357': [0.0, 21.0], 'GSM5988358': [0.0, 51.0], 'GSM5988359': [0.0, 43.0], 'GSM5988360': [0.0, 56.0], 'GSM5988361': [1.0, 26.0], 'GSM5988362': [1.0, 31.0], 'GSM5988363': [1.0, 45.0], 'GSM5988364': [1.0, 39.0], 'GSM5988365': [1.0, 38.0], 'GSM5988366': [1.0, 47.0], 'GSM5988367': [1.0, 29.0], 'GSM5988368': [1.0, 31.0], 'GSM5988369': [1.0, 39.0], 'GSM5988370': [1.0, 37.0], 'GSM5988371': [1.0, 35.0], 'GSM5988372': [1.0, 34.0], 'GSM5988373': [1.0, 50.0], 'GSM5988374': [1.0, 25.0], 'GSM5988375': [1.0, 27.0], 'GSM5988376': [1.0, 38.0], 'GSM5988377': [1.0, 40.0], 'GSM5988378': [1.0, 27.0], 'GSM5988379': [1.0, 44.0], 'GSM5988380': [1.0, 40.0], 

### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506',
       'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529', 'A_19_P00315541'],
      dtype='object', name='ID')

Gene data dimensions: 58341 genes × 48 samples


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers shown above, I can see they do not appear to be standard human gene symbols.
# The identifiers starting with "(+)E1A_r60_" and "A_19_P..." are likely probe IDs from a microarray platform.
# Standard human gene symbols would typically be like BRCA1, TP53, TNF, etc.
# These appear to be Agilent microarray probe IDs that need to be mapped to standard gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate columns from the gene annotation data
# The 'ID' column in the gene annotation matches the index in gene_data (probe identifiers)
# The 'GENE_SYMBOL' column contains the gene symbols we want to map to
print("\nIdentifying probe ID to gene symbol mapping columns...")

# 2. Get gene mapping dataframe using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Generated mapping dataframe with dimensions: {gene_mapping.shape}")
print("First few rows of the mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression
# This function will handle the many-to-many relationships as specified
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"\nConverted gene expression data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("\nFirst 10 gene symbols in the converted data:")
print(gene_data.index[:10])

# 4. Normalize gene symbols to handle synonyms and aggregate duplicate genes
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("\nSample of normalized gene symbols:")
print(gene_data.index[:10])

# Save the processed gene data to a CSV file
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")



Identifying probe ID to gene symbol mapping columns...
Generated mapping dataframe with dimensions: (48862, 2)
First few rows of the mapping:
               ID    Gene
3   A_33_P3396872   CPED1
4   A_33_P3267760    BCOR
5    A_32_P194264   CHAC2
6    A_23_P153745   IFI30
10  A_21_P0014180  GPR146



Converted gene expression data dimensions: 29222 genes × 48 samples

First 10 gene symbols in the converted data:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A1CF-2', 'A1CF-3', 'A2M', 'A2M-1',
       'A2M-AS1', 'A2ML1', 'A2MP1'],
      dtype='object', name='Gene')

Normalizing gene symbols...
After normalization: 20778 genes × 48 samples

Sample of normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A3GALT2', 'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')



Gene expression data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/gene_data/GSE199841.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Load the clinical data previously saved
# Since the gene_data is already normalized in Step 6, we can proceed with linking

# Load the clinical data from Step 2
clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data shape: {clinical_features.shape}")
print(f"Clinical data columns: {clinical_features.columns.tolist()}")
print(f"First few rows of clinical data:")
print(clinical_features.head())

# Load the gene data (already normalized in Step 6)
print(f"Gene data shape: {gene_data.shape}")
print(f"First 5 gene symbols: {gene_data.index[:5].tolist()}")

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Handle missing values
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Determine if trait is biased
print("\nChecking for bias in the trait variable:")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# Conduct final quality validation
is_trait_available = True  # We confirmed trait data is available from Step 2
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data for PTSD patients and healthy controls. All subjects are female."
)

# Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for trait association studies, linked data not saved.")

Loaded clinical data shape: (2, 48)
Clinical data columns: ['GSM5988345', 'GSM5988346', 'GSM5988347', 'GSM5988348', 'GSM5988349', 'GSM5988350', 'GSM5988351', 'GSM5988352', 'GSM5988353', 'GSM5988354', 'GSM5988355', 'GSM5988356', 'GSM5988357', 'GSM5988358', 'GSM5988359', 'GSM5988360', 'GSM5988361', 'GSM5988362', 'GSM5988363', 'GSM5988364', 'GSM5988365', 'GSM5988366', 'GSM5988367', 'GSM5988368', 'GSM5988369', 'GSM5988370', 'GSM5988371', 'GSM5988372', 'GSM5988373', 'GSM5988374', 'GSM5988375', 'GSM5988376', 'GSM5988377', 'GSM5988378', 'GSM5988379', 'GSM5988380', 'GSM5988381', 'GSM5988382', 'GSM5988383', 'GSM5988384', 'GSM5988385', 'GSM5988386', 'GSM5988387', 'GSM5988388', 'GSM5988389', 'GSM5988390', 'GSM5988391', 'GSM5988392']
First few rows of clinical data:
                                GSM5988345  GSM5988346  GSM5988347  \
Post-Traumatic_Stress_Disorder         0.0         0.0         0.0   
Age                                   55.0        53.0        22.0   

                        

Data shape after handling missing values: (48, 20780)

Checking for bias in the trait variable:
For the feature 'Post-Traumatic_Stress_Disorder', the least common label is '0.0' with 16 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Post-Traumatic_Stress_Disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 27.0
  50% (Median): 38.5
  75%: 44.25
Min: 21.0
Max: 59.0
The distribution of the feature 'Age' in this dataset is fine.



Linked data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/GSE199841.csv
